In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple as nt
from matplotlib import gridspec
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.datasets import cifar10 
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

val_samples=5000

Data = nt("Data", "x_train y_train x_valid y_valid x_test y_test")
%matplotlib inline

In [ ]:
# Load data
(x_train, y_train), (x_test, y_test)= cifar10.load_data ()
data = Data(x_train, y_train, None, None, x_test, y_test)

In [ ]:
def visualize_data(data):
    images_to_show = 36
    per_row = 12
    fig = plt.figure(figsize=(20,5))
    for i in range(images_to_show):
        pos = (i // per_row, ((i % per_row) + per_row) % per_row)
        ax = plt.subplot2grid((int(images_to_show / per_row), per_row), pos, xticks=[], yticks=[])
        ax.imshow(np.squeeze(data.x_train[i]))
    plt.show()

In [ ]:
# A chart showing how the accuracy for the training and tests sets evolved
def visualize_training(cifar10):
    
    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Plot the accuracy for the training and tests sets
    ax1.plot(cifar10.history['accuracy'])
    ax1.plot(cifar10.history['val_accuracy'])
    ax1.set_title('Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epochs')
    ax1.legend(['Training', 'Validation'], loc='lower right')

    # Plot the training vs validation loss
    ax2.plot(cifar10.history['loss'])
    ax2.plot(cifar10.history['val_loss'])
    ax2.set_title('Loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epochs')
    ax2.legend(['Training', 'Validation'], loc='upper right')

    # Adjust the spacing between the subplots
    fig.subplots_adjust(wspace=0.4)

    # Show the figure
    plt.show()

In [ ]:
# Visualize the data
visualize_data(data)

In [ ]:
# Preprocess the data
categories = len(np.unique(data.y_train))
print("Shape of x_train pre-processing: ", data.x_train.shape)
print("Shape of y_train pre-processing: ", data.y_train.shape)
processed_data = cnn_preprocess(data, categories)
print("Shape of x_train post-processing: ", processed_data.x_train.shape)
print("Shape of y_train post-processing: ", processed_data.y_train.shape)
print("Shape of x_valid post-processing: ", processed_data.x_valid.shape)
print("Shape of y_valid post-processing: ", processed_data.y_valid.shape)
print("Shape of x_test post-processing: ", processed_data.x_test.shape)
print("Shape of y_test post-processing: ", processed_data.y_test.shape)

In [ ]:
#**CNN***CNN***CNN***CNN**#

In [ ]:
# Z-score normalization
def cnn_preprocess(data, categories):
    
    # Z-score normalization of training data
    mean = np.mean(data.x_train, axis=(0,1,2,3))
    std = np.std(data.x_train, axis=(0,1,2,3))
    x_train = ((data.x_train - mean) / (std + 1e-7)).astype("float32")

    # Z-score normalization of test data
    mean = np.mean(data.x_test, axis=(0,1,2,3))
    std = np.std(data.x_test, axis=(0,1,2,3))
    x_test = ((data.x_test - mean) / (std + 1e-7)).astype("float32")

    y_train = to_categorical(data.y_train, categories)
    y_test = to_categorical(data.y_test, categories)    
    return Data(x_train[val_samples:], y_train[val_samples:],
                x_train[:val_samples], y_train[:val_samples],
                x_test, y_test)

In [ ]:
# Define configuration parameters
start_lr = 0.001
exp_decay = 0.001

# Define the scheduling function
def schedule(epoch):
  def lr(epoch, start_lr, exp_decay):
    return start_lr * math.exp(-exp_decay*epoch)
  return lr(epoch, start_lr, exp_decay)

In [ ]:
def build_cnn(data, categories):
    # Create model architecture
    weight_decay = 1e-4
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, padding="same", activation="elu",
                     kernel_regularizer=regularizers.l2(weight_decay), input_shape=data.x_train.shape[1:]))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size=3, padding="same", activation="elu",
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, kernel_size=3, padding="same", activation="elu",
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=3, padding="same", activation="elu",
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, kernel_size=3, padding="same", activation="elu",
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=3, padding="same", activation="elu",
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.4))
    
    model.add(Flatten())
    model.add(Dense(categories, activation="softmax"))
    
    optimized_rmsprop = RMSprop(learning_rate=0.001,decay=1e-6)
    model.compile(loss="categorical_crossentropy", optimizer=optimized_rmsprop, metrics=["accuracy"])
    return model


In [ ]:
# Preprocess for cnn
cnn_processed_data = cnn_preprocess(data, categories)

In [ ]:
# Build cnn
cnn = build_cnn(cnn_processed_data, categories)
print("CNN architecture:")
cnn.summary()

In [ ]:
# Perform data augmentation
datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.15, height_shift_range=0.15, horizontal_flip=True)
datagen.fit(cnn_processed_data.x_train)

In [ ]:
# Train the optimized cnn
batch_size = 64
cnn_path_best = "weights/cifar10_cnn_best.hdf5"
checkpointer_cnn = ModelCheckpoint(cnn_path_best, verbose=1, save_best_only=True)
cifar10_cnn = cnn.fit(datagen.flow(cnn_processed_data.x_train, cnn_processed_data.y_train, batch_size=batch_size),
                                                 steps_per_epoch=cnn_processed_data.x_train.shape[0] // batch_size, epochs=200,
                                                 verbose=0,validation_data=(cnn_processed_data.x_valid, cnn_processed_data.y_valid),
                                                 callbacks=[checkpointer_cnn, LearningRateScheduler(schedule),
                                                            EarlyStopping(min_delta=0.001, patience=30)])
cnn.load_weights(cnn_path_best)
score_cnn = cnn.evaluate(cnn_processed_data.x_test, cnn_processed_data.y_test, verbose=0)

In [ ]:
visualize_training(cifar10_cnn)

In [ ]:
print("Accuracy cnn: {0:.2f}%".format(score_cnn[1] * 100))

In [ ]:
#**MLP***MLP***MLP***MLP**#

In [ ]:
# Preprocess the data
categories = len(np.unique(data.y_train))
print("Shape of x_train pre-processing: ", data.x_train.shape)
print("Shape of y_train pre-processing: ", data.y_train.shape)
processed_data = mlp_preprocess(data, categories)
print("Shape of x_train post-processing: ", processed_data.x_train.shape)
print("Shape of y_train post-processing: ", processed_data.y_train.shape)
print("Shape of x_valid post-processing: ", processed_data.x_valid.shape)
print("Shape of y_valid post-processing: ", processed_data.y_valid.shape)
print("Shape of x_test post-processing: ", processed_data.x_test.shape)
print("Shape of y_test post-processing: ", processed_data.y_test.shape)

In [ ]:
def mlp_preprocess(data, categories):
    x_train = data.x_train.astype("float32") / 255
    x_test = data.x_test.astype("float32") / 255
    y_train = to_categorical(data.y_train, categories)
    y_test = to_categorical(data.y_test, categories)    
    return Data(x_train[5000:], y_train[5000:],
                x_train[:5000], y_train[:5000],
                x_test, y_test)

In [ ]:
def build_mlp(data, categories):
    # Create model architecture
    model = Sequential()
    model.add(Flatten(input_shape=data.x_train.shape[1:]))
    model.add(Dense(1000, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(categories, activation="softmax"))
    
    # Compile the model
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    return model

In [ ]:
# Build mlp
mlp = build_mlp(processed_data, categories)
print("MLP architecture:")
mlp.summary()

In [ ]:
mlp_weights_path = "weights/cifar10_mlp_best.hdf5"
# Train the mlp
checkpointer_mlp = ModelCheckpoint(filepath=mlp_weights_path, verbose=0, save_best_only=True)
cifar10_mlp = mlp.fit(processed_data.x_train, processed_data.y_train, batch_size=32, 
                   epochs=100, validation_data=(processed_data.x_valid, processed_data.y_valid),
                   callbacks=[checkpointer_mlp], shuffle=True)
mlp.load_weights(mlp_weights_path)
score_mlp = mlp.evaluate(processed_data.x_test, processed_data.y_test, verbose=0)

In [ ]:
visualize_training(cifar10_mlp)

In [ ]:
print("Accuracy mlp: {0:.2f}%".format(score_mlp[1] * 100))